# Exective summary of Work Package 3

## Objectives

In this WP, you will work on a given training dataset. Your goal is to develop a fault detection model using the regression algorithms learnt in the class, in order to achieve best F1 score。

## Tasks

- Task 1: Develop a regression model to predict the reference value for motor temperature.
- Task 2: Develop a fault detection model using the regression model you developed in Task 1.

## Delierables

- A Jupyter notebook reporting the process and results of the above tasks


# Before starting, please:
- Fetch the most up-to-date version of the github repository.
- Create a new branch with your name, based on the "main" branch and switch to your own branch.
- Copy this notebook to the work space of your group, and rename it to TD_WP_3_Your name.ipynb
- After finishing this task, push your changes to the github repository of your group.

# Task 1: Predict normal behaviors through regression models

In this task, let us try to develop a best regression model to predict the normal behaviors of a given motor. In this exercise, we can use motor 6 as an example. You can easilily generate the approach to other models for the data challenge.

We can use all the dataset where motor 6 works normally as our dataset. Then, we can run a cross validation (based on sequence, not points) to test the performances of the developed model.

In this example, we mainly use the following performance metrics:
- max error: The max error between the predicted and the true values.
- Mean root squared error: The mean root squared error between the predicted and the true values.
- Out-of-boundary rate: The percentage that the residual error between the predicted and the true values is larger than a given threshold. Here, we set the thresold to be $3$ degree.

## Sub-task 1: Only use the features at the current moment.

[This notebook](demo_regression_mdl.ipynb) provides a basic demonstration of how to set up the experiment. Let us start by considering only using the features from the current moment. In the notebook, we show a baseline using a simple linear regression with all the features. Could you please try to improve the performance of the model?

A few possible directions:
- Feature selection?
- Smoothing?
- Removing sequence-to-sequence variablity? Adding features regarding time dynamics (see the TD for last lecture).
- Changing to other regression models? For this, you can try different regression models from [here](https://scikit-learn.org/stable/supervised_learning.html)

Put your code below.

In [2]:
# Import the libraries

import sys
sys.path.insert(0, 'C:/Users/gioca/GitHub/projects/maintenance_industry_4_2024/supporting_scripts/WP_2_20240516/')
from utility import read_all_test_data_from_path, extract_selected_feature, prepare_sliding_window, FaultDetectReg, read_all_csvs_one_test, run_cv_one_motor
import numpy as np
from numpy import where
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import LocalOutlierFactor


In [3]:
# Data Reading and Outlier Removal
base_dictionary = 'C:/Users/gioca/GitHub/projects/maintenance_industry_4_2024/dataset/training_data/'
df_data = read_all_test_data_from_path(base_dictionary, is_plot=False)

normal_test_id = ['20240105_164214', '20240105_165300', '20240105_165972', '20240320_152031', '20240320_153841',
                  '20240320_155664', '20240321_122650', '20240325_135213', '20240426_141190', '20240426_141532',
                  '20240426_141602', '20240426_141726', '20240426_141938', '20240426_141980', '20240503_164435']

df_data = df_data[df_data['test_condition'].isin(normal_test_id)]

def removeoutliers (df, experiment, feature):

    model = LocalOutlierFactor(n_neighbors=20)
    df['indice'] = df.reset_index().index
    ds = df.loc[df['test_condition'] == experiment]
    x = pd.DataFrame(ds, columns=['indice',feature])
    model.fit_predict(x)
    lof = model.negative_outlier_factor_ 
    df_out = pd.DataFrame({'indice': x['indice'], 'lof': lof })

    return df_out

features = ['data_motor_1_position', 'data_motor_1_temperature', 'data_motor_1_voltage', 
            'data_motor_2_position', 'data_motor_2_temperature', 'data_motor_2_voltage',
            'data_motor_3_position', 'data_motor_3_temperature', 'data_motor_3_voltage',
            'data_motor_4_position', 'data_motor_4_temperature', 'data_motor_4_voltage',
            'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_5_voltage',
            'data_motor_6_position', 'data_motor_6_temperature', 'data_motor_6_voltage']

for i in range(len(normal_test_id)):
    for j in range(len(features)):
        outliers = removeoutliers(df_data, normal_test_id[i], features[j])
        thresh1 = np.quantile(outliers.lof, 1/3000)
        index = where(outliers.lof<=thresh1)
        final_index = np.ravel(outliers['indice'].iloc[0] + index)
        df_data = df_data[~df_data['indice'].isin(final_index)]

In [4]:
# Definition of the Pipeline
steps = [
    ('standardizer', StandardScaler()),  # Step 1: StandardScaler
    ('regressor', LinearRegression())    # Step 2: Linear Regression
]

# Create the pipeline
mdl_linear_regreession = Pipeline(steps)

In [5]:
# Model 1: all features
threshold = 1
df_all_1 = run_cv_one_motor(motor_idx=6, df_data=df_data, mdl=mdl_linear_regreession, feature_list=features, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0  13.123969  4.521851              0.971376
1  12.759905  4.033491              0.990476
2  25.193219  4.232480              0.636437
3   6.871982  3.701900              0.756065
4  10.669683  7.021780              0.874039


Mean performance metric and standard error:
Max error: 13.7238 +- 6.8759
RMSE: 4.7023 +- 1.3305
Exceed boundary rate: 0.8457 +- 0.1495




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [39]:
# Model 2: feature selection based on correlation matrix

X_data = pd.DataFrame(df_data, columns=features)
correlation_matrix = X_data.corr()
# plt.figure(figsize=(10, 10))  # Adjust width and height as needed
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
threshold = 0.25
selected_features_1 = correlation_matrix.columns[correlation_matrix.loc['data_motor_6_temperature'].abs() > threshold].tolist()

threshold = 1
df_sel_1 = run_cv_one_motor(motor_idx=6, df_data=df_data, mdl=mdl_linear_regreession, feature_list=selected_features_1, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0   4.852029  2.130532              0.843801
1   8.790258  3.273211              0.997143
2  53.200395  3.754411              0.825208
3   6.705503  3.721872              0.832884
4   8.178076  4.778860              0.860438


Mean performance metric and standard error:
Max error: 16.3453 +- 20.6585
RMSE: 3.5318 +- 0.9580
Exceed boundary rate: 0.8719 +- 0.0713




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [7]:
# Model 3: voltage features excluded

features_no_voltage = ['data_motor_1_position', 'data_motor_1_temperature', 'data_motor_2_position', 'data_motor_2_temperature',
                       'data_motor_3_position', 'data_motor_3_temperature', 'data_motor_4_position', 'data_motor_4_temperature',
                       'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_6_position', 'data_motor_6_temperature']

threshold = 1
df_no_voltage_1 = run_cv_one_motor(motor_idx=6, df_data=df_data, mdl=mdl_linear_regreession, feature_list=features_no_voltage, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0   9.045765  4.386853              0.975700
1  14.139928  4.786639              0.999048
2  18.810248  4.028403              0.750693
3   6.994967  4.473019              0.754717
4   8.126197  4.791908              0.857481


Mean performance metric and standard error:
Max error: 11.4234 +- 4.9518
RMSE: 4.4934 +- 0.3174
Exceed boundary rate: 0.8675 +- 0.1178




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [8]:
# Smoothing

df_smoothing = df_data.copy()

def smoothing(data, experiment, feature, dim):

    ds = data.loc[data['test_condition'] == experiment]
    ds = pd.DataFrame(ds, columns=['time', 'indice', feature])

    ds['aver'] = ds[feature].rolling(dim).mean()

    dict = ds.set_index('indice')['aver'].to_dict()
    for idx, row in data.iterrows():
     if row['indice'] in dict:
        data.at[idx, feature] = dict[row['indice']]

    return data

for i in range(len(normal_test_id)):
   for j in range(len(features)):
     df_smoothing = smoothing(df_smoothing, normal_test_id[i], features[j], 25)

df_smoothing = df_smoothing.dropna()


In [9]:
# Model 4: all features, data smoothing

threshold = 1
df_all_2 = run_cv_one_motor(motor_idx=6, df_data=df_smoothing, mdl=mdl_linear_regreession, feature_list=features, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0  11.234058  4.052878              0.932780
1   7.431778  4.517819              0.997955
2  22.769134  3.197044              0.824674
3   5.396482  2.830593              0.832836
4  11.373792  7.575421              0.914762


Mean performance metric and standard error:
Max error: 11.6410 +- 6.7228
RMSE: 4.4348 +- 1.8789
Exceed boundary rate: 0.9006 +- 0.0726




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [38]:
# Model 5: feature selection based on correlation matrix, data smoothing

X_data = pd.DataFrame(df_smoothing, columns=features)
correlation_matrix = X_data.corr()
# plt.figure(figsize=(10, 10))  # Adjust width and height as needed
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
threshold = 0.25
selected_features_2 = correlation_matrix.columns[correlation_matrix.loc['data_motor_6_temperature'].abs() > threshold].tolist()

threshold = 1
df_sel_2 = run_cv_one_motor(motor_idx=6, df_data=df_smoothing, mdl=mdl_linear_regreession, feature_list=selected_features_2, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0   5.064896  1.707754              0.553734
1   7.875447  3.315229              0.930470
2  30.841097  4.771816              0.901095
3   7.080855  4.186829              0.973134
4   8.806041  5.605110              0.892526


Mean performance metric and standard error:
Max error: 11.9337 +- 10.6592
RMSE: 3.9173 +- 1.4913
Exceed boundary rate: 0.8502 +- 0.1687




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [11]:
# Model 6: voltage features excluded, data smoothing

features_no_voltage = ['data_motor_1_position', 'data_motor_1_temperature', 'data_motor_2_position', 'data_motor_2_temperature',
                       'data_motor_3_position', 'data_motor_3_temperature', 'data_motor_4_position', 'data_motor_4_temperature',
                       'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_6_position', 'data_motor_6_temperature']

threshold = 1
df_no_voltage_2 = run_cv_one_motor(motor_idx=6, df_data=df_smoothing, mdl=mdl_linear_regreession, feature_list=features_no_voltage, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0   8.550900  4.464727              0.974274
1   8.910532  6.313933              1.000000
2   7.194869  4.036268              0.760834
3   7.248788  4.774894              0.759701
4   7.715723  4.771697              0.854231


Mean performance metric and standard error:
Max error: 7.9242 +- 0.7741
RMSE: 4.8723 +- 0.8607
Exceed boundary rate: 0.8698 +- 0.1141




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
# Data preprocessing (prof function)

from scipy.signal import butter, filtfilt

# Function to design a Butterworth low-pass filter
def butter_lowpass(cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

# Function to apply the Butterworth low-pass filter
def lowpass_filter(data, cutoff_freq, sampling_freq, order=5):
    b, a = butter_lowpass(cutoff_freq, sampling_freq, order=order)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

# Set parameters for the low-pass filter
cutoff_frequency = .5  # Adjust as needed
sampling_frequency = 10  # Assuming your data is evenly spaced in time

def customized_outlier_removal(df: pd.DataFrame):
    ''' # Description
    Remove outliers from the dataframe based on defined valid ranges. 
    Define a valid range of temperature and voltage. 
    Use ffil function to replace the invalid measurement with the previous value.
    '''
    df['position'] = df['position'].where(df['position'] <= 1000, np.nan)
    df['position'] = df['position'].where(df['position'] >= 0, np.nan)
    df['position'] = df['position'].ffill()
    df['position'] = lowpass_filter(df['position'], cutoff_frequency, sampling_frequency)
    df['position'] = df['position'].rolling(window=10, min_periods=1).mean()
    df['position'] = df['position'].round()

    df['temperature'] = df['temperature'].where(df['temperature'] <= 100, np.nan)
    df['temperature'] = df['temperature'].where(df['temperature'] >= 0, np.nan)
    df['temperature'] = df['temperature'].rolling(window=10, min_periods=1).mean()

    # Make sure that the difference between the current and previous temperature cannot be too large.
    # Define your threshold
    threshold = 10
    # Shift the 'temperature' column by one row to get the previous temperature
    prev_tmp = df['temperature'].shift(1)
    # Calculate the absolute difference between current and previous temperature
    temp_diff = np.abs(df['temperature'] - prev_tmp)
    # Set the temperature to NaN where the difference is larger than the threshold
    df.loc[temp_diff > threshold, 'temperature'] = np.nan
    df['temperature'] = df['temperature'].ffill()

    df['voltage'] = df['voltage'].where(df['voltage'] >= 6000, np.nan)
    df['voltage'] = df['voltage'].where(df['voltage'] <= 8000, np.nan)
    df['voltage'] = df['voltage'].ffill()
    df['voltage'] = lowpass_filter(df['voltage'], cutoff_frequency, sampling_frequency)
    df['voltage'] = df['voltage'].rolling(window=5, min_periods=1).mean()
    
from utility import read_all_csvs_one_test
import matplotlib.pyplot as plt
import os

# Get all the folders in the base_dictionary
path_list = os.listdir(base_dictionary)
# Only keep the folders, not the excel file.
path_list = path_list[:-1]

# Read the data.
df_data_smooth = pd.DataFrame()
for tmp_path in path_list:
    path = base_dictionary + tmp_path
    # Read the data with the customized outlier removal function.
    tmp_df = read_all_csvs_one_test(path, tmp_path, customized_outlier_removal)
    df_data_smooth = pd.concat([df_data_smooth, tmp_df])
    df_data_smooth = df_data_smooth.reset_index(drop=True)

df_data_smooth = df_data_smooth[df_data_smooth['test_condition'].isin(normal_test_id)]

In [17]:
# Model 7: all features, data smoothing (prof function)

threshold = 1
df_all_3 = run_cv_one_motor(motor_idx=6, df_data=df_data_smooth, mdl=mdl_linear_regreession, feature_list=features, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0  13.750723  2.979289              0.856284
1  10.427738  5.119518              0.983005
2   6.817184  3.274084              0.813810
3   9.096755  3.665289              0.701493
4  14.917425  6.741679              0.911580


Mean performance metric and standard error:
Max error: 11.0020 +- 3.3301
RMSE: 4.3560 +- 1.5666
Exceed boundary rate: 0.8532 +- 0.1059




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [37]:
# Model 8: feature selection with correlation matrix, data smoothing (prof function)

X_data = pd.DataFrame(df_data_smooth, columns=features)
correlation_matrix = X_data.corr()
# plt.figure(figsize=(10, 10))  # Adjust width and height as needed
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
threshold = 0.25
selected_features_3 = correlation_matrix.columns[correlation_matrix.loc['data_motor_6_temperature'].abs() > threshold].tolist()

threshold = 1
df_sel_3 = run_cv_one_motor(motor_idx=6, df_data=df_data_smooth, mdl=mdl_linear_regreession, feature_list=selected_features_3, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0   9.799860  1.547442              0.485170
1  10.737072  3.281197              0.889982
2  10.142434  3.767738              0.706401
3  10.477643  4.485158              0.949005
4  13.682427  5.568414              0.923560


Mean performance metric and standard error:
Max error: 10.9679 +- 1.5579
RMSE: 3.7300 +- 1.4930
Exceed boundary rate: 0.7908 +- 0.1956




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [19]:
# Model 9: voltage features excluded, data smoothing (prof function)

features_no_voltage = ['data_motor_1_position', 'data_motor_1_temperature', 'data_motor_2_position', 'data_motor_2_temperature',
                       'data_motor_3_position', 'data_motor_3_temperature', 'data_motor_4_position', 'data_motor_4_temperature',
                       'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_6_position', 'data_motor_6_temperature']

threshold = 1
df_no_voltage_3 = run_cv_one_motor(motor_idx=6, df_data=df_data_smooth, mdl=mdl_linear_regreession, feature_list=features_no_voltage, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0   8.421144  4.166940              0.976149
1   9.366627  6.378738              1.000000
2   6.993277  3.917188              0.775907
3   7.188595  4.570516              0.753731
4   8.316629  4.795391              0.822019


Mean performance metric and standard error:
Max error: 8.0573 +- 0.9748
RMSE: 4.7658 +- 0.9643
Exceed boundary rate: 0.8656 +- 0.1148




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [29]:
# Sequence-To-Sequence Variability

def compensate_seq_bias(df: pd.DataFrame):
    ''' # Description
    Adjust for the sequence-to-sequence bias.
    '''
    # Tranform the features relative to the first data point.
    df['temperature'] = df['temperature'] - df['temperature'].iloc[0]
    df['voltage'] = df['voltage'] - df['voltage'].iloc[0]
    df['position'] = df['position'] - df['position'].iloc[0]

def remove_outliers(df: pd.DataFrame):
    ''' # Description
    Remove outliers from the dataframe based on defined valid ranges. 
    Define a valid range of temperature and voltage. 
    Use ffil function to replace the invalid measurement with the previous value.
    '''
    df['temperature'] = df['temperature'].where(df['temperature'] <= 100, np.nan)
    df['temperature'] = df['temperature'].where(df['temperature'] >= 0, np.nan)
    df['temperature'] = df['temperature'].ffill()        

    df['voltage'] = df['voltage'].where(df['voltage'] >= 6000, np.nan)
    df['voltage'] = df['voltage'].where(df['voltage'] <= 9000, np.nan)
    df['voltage'] = df['voltage'].ffill()        

    df['position'] = df['position'].where(df['position'] >= 0, np.nan)
    df['position'] = df['position'].where(df['position'] <= 1000, np.nan)
    df['position'] = df['position'].ffill()


# Subfunction for data preprocessing.
def pre_processing(df: pd.DataFrame):
    ''' ### Description
    Preprocess the data:
    - remove outliers
    - Adjust for the sequence-to-sequence bias.
    - add new features about the difference between the current and previous n data point.
    '''     
    # Start processing.
    remove_outliers(df)
    compensate_seq_bias(df)

df_data_sts = read_all_test_data_from_path(base_dictionary, pre_processing, is_plot=False)
df_data_sts = df_data_sts[df_data_sts['test_condition'].isin(normal_test_id)]
df_data_sts['indice'] = df_data_sts.reset_index().index

for i in range(len(normal_test_id)):
   for j in range(len(features)):
     df_data_sts = smoothing(df_data_sts, normal_test_id[i], features[j], 25)

df_data_sts = df_data_sts.dropna()

In [30]:
# Model 10: all features, sequence-to-sequence variability

threshold = 1
df_all_smooth_4 = run_cv_one_motor(motor_idx=6, df_data=df_data_sts, mdl=mdl_linear_regreession, feature_list=features, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0  12.700275  5.339700              0.962830
1  11.121603  2.158902              0.499044
2  11.451561  5.508573              0.829119
3  11.406325  4.560663              0.857923
4  14.814753  9.382691              0.897085


Mean performance metric and standard error:
Max error: 12.2989 +- 1.5323
RMSE: 5.3901 +- 2.6022
Exceed boundary rate: 0.8092 +- 0.1805




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [36]:
# Model 11: feature selection with correlation matrix, sequence-to-sequence variability

X_data = pd.DataFrame(df_data_sts, columns=features)
correlation_matrix = X_data.corr()
# plt.figure(figsize=(10, 10))  # Adjust width and height as needed
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
threshold = 0.25
selected_features_4 = correlation_matrix.columns[correlation_matrix.loc['data_motor_6_temperature'].abs() > threshold].tolist()

threshold = 1
df_sel_4 = run_cv_one_motor(motor_idx=6, df_data=df_data_sts, mdl=mdl_linear_regreession, feature_list=selected_features_4, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0  10.844163  1.416390              0.286066
1   8.201481  1.305687              0.207457
2  10.460351  3.488934              0.780395
3   4.759013  1.976267              0.465847
4  10.542505  4.682321              0.872100


Mean performance metric and standard error:
Max error: 8.9615 +- 2.5753
RMSE: 2.5739 +- 1.4653
Exceed boundary rate: 0.5224 +- 0.2946




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [32]:
# Model 12: voltage features excluded, sequence-to-sequence variability

features_no_voltage = ['data_motor_1_position', 'data_motor_1_temperature', 'data_motor_2_position', 'data_motor_2_temperature',
                       'data_motor_3_position', 'data_motor_3_temperature', 'data_motor_4_position', 'data_motor_4_temperature',
                       'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_6_position', 'data_motor_6_temperature']

threshold = 1
df_no_voltage_4 = run_cv_one_motor(motor_idx=6, df_data=df_data_sts, mdl=mdl_linear_regreession, feature_list=features_no_voltage, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0  12.383555  3.574975              0.929972
1   4.378222  1.968402              0.410134
2  11.505006  5.834052              0.859601
3   6.408303  2.665664              0.531421
4  13.818204  7.554588              0.982748


Mean performance metric and standard error:
Max error: 9.6987 +- 4.0797
RMSE: 4.3195 +- 2.3225
Exceed boundary rate: 0.7428 +- 0.2557




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


**Summary of the results - Only using features from the current moment**

Please add a table in the end, summarying the results from all the models. Please write a few texts to explain what is the best model you got (including the features and preprocessing you did), its performance, and how could you further improve it.

| Model   | Max error | MRSE | Exceed boundary rate |
|---------|----------|-----------|--------|
| 1. All Features |   13,70 +- 6,83  |   4,70 +- 1,33   |  0,85 +- 0,15 | 
| 2. Selected Features |   16,35 +- 20,66  |   3,53 +- 0,96   |  0,87 +- 0,07 | 
| 3. No Voltage Features |   11,42 +- 4,95  |   4,49 +- 0,32   |  0,87 +- 0,12 | 
| 4. All Features, Smoothing (1) |   11,64 +- 6,72  |   4,43 +- 1,88   |  0,90 +- 0,07 | 
| 5. Selected Features, Smoothing (1) |   11,93 +- 10,66  |   3,92 +- 1,49   |  0,85 +- 0,17 | 
| 6. No Voltage Features, Smoothing (1) |   7,92 +- 0,77  |   4,87 +- 0,86   |  0,87 +- 0,11 | 
| 7. All Features, Smoothing (2) |   11,00 +- 3,33  |   4,36 +- 1,57   |  0,85 +- 0,11 | 
| 8. Selected Features, Smoothing (2) |   10,97 +- 1,56  |   3,73 +- 1,49   |  0,79 +- 0,20 | 
| 9. No Voltage Features, Smoothing (2) |   8,06 +- 0,97  |   4,77 +- 0,96   |  0,87 +- 0,11 | 
| 10. All Features, Smoothing (1), STS Variability |   12,30 +- 1,53  |   5,39 +- 2,60   |  0,81 +- 0,18 | 
| 11. Selected Features, Smoothing (1), STS Variability |   8,97 +- 2,58  |   2,57 +- 1,47   |  0,52 +- 0,30 | 
| 12. No Voltage Features, Smoothing (1), STS Variability |   9,70 +- 4,08  |   4,32 +- 2,32   |  0,74 +- 0,26 | 

## Sub-task 2: Include the features in the past

Now, let's consider using the sliding window approach to include the past in the regression model as well. Please have a look at the demo notebook, run your experiment, and report the best models you could have if you apply the sliding window approach.

In [43]:
# Model 11, window = 5, sample = 1

threshold = 1
window_size = 5
sample_step = 1
prediction_lead_time = 1 # We add the temperature measurement up to 1 point before the current time.

df_sel_4_window_1 = run_cv_one_motor(motor_idx=6, df_data=df_data_sts, mdl=mdl_linear_regreession, feature_list=selected_features_4, 
                            window_size=window_size, sample_step=sample_step, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0   0.077735  0.009400                   0.0
1   0.045861  0.005161                   0.0
2   0.075032  0.006676                   0.0
3   0.048285  0.008286                   0.0
4   0.069005  0.008397                   0.0


Mean performance metric and standard error:
Max error: 0.0632 +- 0.0151
RMSE: 0.0076 +- 0.0017
Exceed boundary rate: 0.0000 +- 0.0000




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [44]:
# Model 11, window = 50, sample = 10

threshold = 1
window_size = 50
sample_step = 10
prediction_lead_time = 1 # We add the temperature measurement up to 1 point before the current time.

df_sel_4_window_2 = run_cv_one_motor(motor_idx=6, df_data=df_data_sts, mdl=mdl_linear_regreession, feature_list=selected_features_4, 
                            window_size=window_size, sample_step=sample_step, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0   0.485490  0.062810                   0.0
1   0.406472  0.078682                   0.0
2   0.584822  0.080506                   0.0
3   0.479464  0.156094                   0.0
4   0.593500  0.134200                   0.0


Mean performance metric and standard error:
Max error: 0.5099 +- 0.0788
RMSE: 0.1025 +- 0.0403
Exceed boundary rate: 0.0000 +- 0.0000




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [46]:
# Model 2, window = 5, sample = 1

threshold = 1
window_size = 5
sample_step = 1
prediction_lead_time = 1 # We add the temperature measurement up to 1 point before the current time.

df_sel_1_window_1 = run_cv_one_motor(motor_idx=6, df_data=df_data, mdl=mdl_linear_regreession, feature_list=selected_features_1, 
                            window_size=window_size, sample_step=sample_step, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0   1.018586  0.165502              0.004639
1   1.002757  0.095527              0.000963
2   1.179037  0.122841              0.003008
3   1.026351  0.146347              0.002740
4   1.021543  0.148666              0.005360


Mean performance metric and standard error:
Max error: 1.0497 +- 0.0729
RMSE: 0.1358 +- 0.0272
Exceed boundary rate: 0.0033 +- 0.0017




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [47]:
# Model 1, window = 5, sample = 1

threshold = 1
window_size = 5
sample_step = 1
prediction_lead_time = 1 # We add the temperature measurement up to 1 point before the current time.

df_all_1_window_1 = run_cv_one_motor(motor_idx=6, df_data=df_data, mdl=mdl_linear_regreession, feature_list=features, 
                            window_size=window_size, sample_step=sample_step, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0   1.038431  0.165787              0.004433
1   1.045970  0.093914              0.002890
2   1.086477  0.123664              0.002198
3   1.033289  0.144928              0.004110
4   1.337769  0.153866              0.003574


Mean performance metric and standard error:
Max error: 1.1084 +- 0.1299
RMSE: 0.1364 +- 0.0283
Exceed boundary rate: 0.0034 +- 0.0009




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [49]:
# Model 11, window = 5, sample = 3

threshold = 1
window_size = 5
sample_step = 3
prediction_lead_time = 1 # We add the temperature measurement up to 1 point before the current time.

df_sel_4_window_1 = run_cv_one_motor(motor_idx=6, df_data=df_data_sts, mdl=mdl_linear_regreession, feature_list=selected_features_4, 
                            window_size=window_size, sample_step=sample_step, n_fold=5, threshold=threshold, mdl_type='reg', single_run_result=False)

Model for motor 6:


c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared

   Max error      RMSE  Exceed boundary rate
0   0.195311  0.028188                   0.0
1   0.294448  0.041021                   0.0
2   0.261043  0.042239                   0.0
3   0.279116  0.077031                   0.0
4   0.238772  0.060632                   0.0


Mean performance metric and standard error:
Max error: 0.2537 +- 0.0387
RMSE: 0.0498 +- 0.0191
Exceed boundary rate: 0.0000 +- 0.0000




c:\Users\gioca\GitHub\.conda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


**Summary of the results - Sliding window**

Please add a table in the end, summarying the results from all the models. Please write a few texts to explain what is the best model you got (including the features and preprocessing you did), its performance, and how could you further improve it.

| Model   (also report parameters like window_size, sample_step, prediction_lead_time, etc.) | Max error | MRSE | Exceed boundary rate |
|---------|----------|-----------|--------|
| Model 1 |   XX.X%  |   XX.X%   |  XX.X% | 
| Model 2 |   XX.X%  |   XX.X%   |  XX.X% | 
| Model 3 |   XX.X%  |   XX.X%   |  XX.X% | 

# Task 2 Fault detection based on regression model

In this exercise, we use the dataset that contains the failure of motor 6 to test the fault detection model based on the regression model trained before. 

[This notebook](demo_FaultDetectReg.ipynb) presents a demonstration of how to use the provided supporting function to develop fault detection model based on the regression model. Please have a look at this notebook, and try to improve the performance of the baseline model.

In [ ]:
# Your code here:

## Summary of the results

Please add a table in the end, summarying the results from all the models (including the unsupervised learning models). Please write a few texts to explain what is the best model you got (including key parameters like threshold, window_size, sample_step, prediction_lead_time, etc), its performance, and how could you further improve it.

| Model   | Accuracy | Precision | Recall | F1   |
|---------|----------|-----------|--------|------|
| Model 1 |   XX.X%  |   XX.X%   |  XX.X% | XX.X%|
| Model 2 |   XX.X%  |   XX.X%   |  XX.X% | XX.X%|
| Model 3 |   XX.X%  |   XX.X%   |  XX.X% | XX.X%|
